In [1]:
import sys
parent_dir='C:\\Users\\garav\\AGOP\\DLR'
#parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
from utils import trainer as t
from torch.utils.data import Dataset
import random
import torch.backends.cudnn as cudnn
import rfm
import numpy as np
import visdom
import model1
import numpy as np
from sklearn.model_selection import train_test_split
from torch.linalg import norm


In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda:0


In [10]:
pwd

'C:\\Users\\garav\\AGOP\\DLR\\experiments\\rfm_mnist'

In [7]:
def one_hot_data(dataset, num_samples=-1):
    labelset = {}
    for i in range(10):
        one_hot = torch.zeros(10)
        one_hot[i] = 1
        labelset[i] = one_hot

    subset = [(ex.flatten(), labelset[label]) for \
              idx, (ex, label) in enumerate(dataset) if idx < num_samples]
    return subset


def group_by_class(dataset):
    labelset = {}
    for i in range(10):
        labelset[i] = []
    for i, batch in enumerate(dataset):
        img, label = batch
        labelset[label].append(img.view(1, 3, 32, 32))
    return labelset


def split(trainset, p=.8):
    train, val = train_test_split(trainset, train_size=p)
    return train, val


In [11]:
torch.cuda.empty_cache()

In [ ]:

SEED = 5636
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)
#torch.cuda.manual_seed(SEED)
cudnn.benchmark = False

transform = transforms.Compose(
        [transforms.ToTensor()
        ])

mnist_transform = transforms.Compose(
        [transforms.Resize([32,32]),
         transforms.ToTensor(),
         transforms.Lambda(lambda x: x.repeat(3, 1, 1))
        ])


path = '~/data/'
    
mnist_trainset = torchvision.datasets.MNIST(root=path,
                                                train=True,
                                                transform=mnist_transform,
                                                download=True)

trainset, valset = split(mnist_trainset, p=.8)
print("Train Size: ", len(trainset), "Val Size: ", len(valset))

trainloader = torch.utils.data.DataLoader(trainset, batch_size=100,
                                              shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(valset, batch_size=100,
                                            shuffle=False, num_workers=1)


mnist_testset = torchvision.datasets.MNIST(root=path,
                                               train=False,
                                               transform=mnist_transform,
                                               download=True)

print("Test Size: ", len(mnist_testset))

testloader = torch.utils.data.DataLoader(mnist_testset, batch_size=128,
                                             shuffle=False, num_workers=2)

name = 'mnist'


In [17]:
print(mnist_testset[0].shape)

AttributeError: 'tuple' object has no attribute 'shape'

In [19]:
net = model1.Net(32, num_classes=10)

In [14]:
t.train_network(trainloader, valloader, testloader,
                    num_classes=10,
                    name=name,model=net)

NameError: name 'trainloader' is not defined